# Chapter 6 Linear Model Selections and Regularization

The Ordinary Least Squares Linear Regression modelis great in theory, and even in many applications, however, with the growing amounts of data available, and particularly in high-dimensional datasets, OLS quickly becomes impractical. There is still much value in linear models and this chapter discusses methods of extending the linear model's reach.

* **Subset Selection**: identify a subset of predictors, and build model from that subset
    * Best subset selection
    * Forward stepwise selection
    * Backward stepwise selection
* **Shrinkage **: fit a model with all predictors, but reduce variance of coefficients by shrinking (regularizing) the coefficients towards zero 
    * Ridge Regression
    * Lasso Regression
* **Dimension Reduction**: project $p$ predictors into $M$ dimensions, where $M<p$, by computing $M$ different linear combinations of the variables.
    * Principal Component Analysis
    * Partial Least Squares

[Analytics Vidhya Ridge Regression Tutorial](https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/)

## Lab 2: Ridge Regression and the Lasso

In [76]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn.linear_model as skl_lm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

from sklearn.model_selection import KFold, cross_val_score
#from sklearn.linear_model import Ridge, RidgeCV

In [29]:
hitters = pd.read_csv('Data/Hitters.csv', index_col=0).dropna()

In [30]:
hitters.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [31]:
dummies = pd.get_dummies(hitters[['League', 'Division', 'NewLeague']])
dummies.info()
print(dummies.head())

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, -Alan Ashby to -Willie Wilson
Data columns (total 6 columns):
League_A       263 non-null uint8
League_N       263 non-null uint8
Division_E     263 non-null uint8
Division_W     263 non-null uint8
NewLeague_A    263 non-null uint8
NewLeague_N    263 non-null uint8
dtypes: uint8(6)
memory usage: 3.6+ KB
                   League_A  League_N  Division_E  Division_W  NewLeague_A  \
-Alan Ashby               0         1           0           1            0   
-Alvin Davis              1         0           0           1            1   
-Andre Dawson             0         1           1           0            0   
-Andres Galarraga         0         1           1           0            0   
-Alfredo Griffin          1         0           0           1            1   

                   NewLeague_N  
-Alan Ashby                  1  
-Alvin Davis                 0  
-Andre Dawson                1  
-Andres Galarraga            1  
-Al

In [32]:
y = hitters.Salary
# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = hitters.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

In [33]:
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
-Alan Ashby,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
-Alvin Davis,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
-Andre Dawson,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
-Andres Galarraga,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
-Alfredo Griffin,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [34]:
y.head()

-Alan Ashby          475.0
-Alvin Davis         480.0
-Andre Dawson        500.0
-Andres Galarraga     91.5
-Alfredo Griffin     750.0
Name: Salary, dtype: float64

### Ridge Regression

Here, we will fit our Ridge Regression model with 100 different alpha values (it is a bit confusing that in the ISLR book and in R's glmnet() function, they use the variable lambda, but in Scikit-Learn, they use alpha), then determine which corresponds to the most accurate model.

In [35]:
# Standardize our Variables
scaler = StandardScaler().fit(X)


alphas = 10**np.linspace(10,-2,100)
ridge = skl_lm.Ridge()
coefs = []

for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(scaler.transform(X), y)
    coefs.append(ridge.coef_)

In [36]:
len(coefs)

100

In [37]:
scaler = StandardScaler()
def Ridge_Regression(X, y, alpha):
    scaler = StandardScaler()
    ridge = skl_lm.Ridge()
    ridge.set_params(alpha=alpha)
    ridge.fit(scaler.fit_transform(X), y)
    return ridge.coef_

In [38]:
Ridge_Regression(X, y, alpha=705)

array([ 16.43830169,  24.15448953,  11.54113148,  20.8012455 ,
        20.00036698,  23.89155483,  13.47868147,  22.48567718,
        25.855997  ,  23.99588393,  26.46025971,  26.82575006,
        18.86346091,  24.65950143,   1.67820571,  -2.60496312,
         4.29756313, -19.60756959,   3.19229238])

#### RidgeCV to choose the optimal $\lambda$ parameter

In [39]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = pd.read_csv('Data/Hitters_X_train.csv', index_col=0)
y_train = pd.read_csv('Data/Hitters_y_train.csv', index_col=0)
X_test = pd.read_csv('Data/Hitters_X_test.csv', index_col=0)
y_test = pd.read_csv('Data/Hitters_y_test.csv', index_col=0)



ridgecv = skl_lm.RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
ridgecv.fit(scaler.fit_transform(X_train), y_train)

RidgeCV(alphas=array([  1.00000e+10,   7.56463e+09, ...,   1.32194e-02,   1.00000e-02]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring='neg_mean_squared_error', store_cv_values=False)

In [40]:
ridgecv.alpha_

100.0

Now that we have our optimal alpha, ``ridgecv.alpha_=100``, we can plug that into our Ridge Regresssion in order to minimize the Test MSE.

In [41]:
optimal_ridge = skl_lm.Ridge()
optimal_ridge.set_params(alpha=ridgecv.alpha_)
optimal_ridge.fit(scaler.fit_transform(X_train), y_train)

Ridge(alpha=100.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [42]:
mean_squared_error(y_test, optimal_ridge.predict(scaler.fit_transform(X_test)))

97020.517151355409

In [43]:
pd.Series(optimal_ridge.coef_.flatten(), index=X.columns)

AtBat           5.559649
Hits           22.893732
HmRun          19.431685
Runs           19.859330
RBI            21.074810
Walks          58.870478
Years          -6.787442
CAtBat         20.752173
CHits          30.645308
CHmRun         13.816343
CRuns          38.037441
CRBI           20.334967
CWalks         24.582540
PutOuts        16.848101
Assists       -46.965887
Errors         57.761412
League_N        6.213095
Division_W     -0.816456
NewLeague_N    11.254522
dtype: float64

### The Lasso

First, let's find the optimal alpha using cross validation.

In [63]:
lassocv = skl_lm.LassoCV(alphas = alphas, max_iter=10000)
lassocv.fit(scaler.fit_transform(X_train), y_train.values.ravel())

LassoCV(alphas=array([  1.00000e+10,   7.56463e+09, ...,   1.32194e-02,   1.00000e-02]),
    copy_X=True, cv=None, eps=0.001, fit_intercept=True, max_iter=10000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [64]:
lassocv.alpha_

10.72267222010321

In [65]:
lasso = skl_lm.Lasso()
lasso.set_params(alpha = lassocv.alpha_)
lasso.fit(scaler.fit_transform(X_train), y_train.values.ravel())
mean_squared_error(y_test, lasso.predict(scaler.fit_transform(X_test)))

101618.27447998914

As we can see by the Test MSE, Ridge Regression performs slightly better than Lasso on this dataset.

In [66]:
pd.Series(lasso.coef_.flatten(), index = X.columns)

AtBat           -0.000000
Hits             0.351477
HmRun           17.238693
Runs             0.000000
RBI             22.438006
Walks          104.182138
Years          -25.064170
CAtBat           0.000000
CHits            0.000000
CHmRun          -0.000000
CRuns          173.024748
CRBI             0.000000
CWalks          -0.000000
PutOuts         21.238595
Assists        -69.562708
Errors          88.173785
League_N         0.928679
Division_W       0.000000
NewLeague_N      0.000000
dtype: float64

Interestingly, Lasso Regression cancels out many of the predictors, using only 10 of the original 19.

## Lab 3: PCR and PLS Regression (Dimension Reduction)

Principal Components Regression and Partial Least Squares Regression differ from Ordinary Least Squares regression in that instead of making predicting based on a linear combination of predictors, they transform the original predictors $p$ into $M$ predictors where $M<p$, and then use a linear combination of the $M$ predictors to fit a least squares model.

Expressed mathematically,
$$\sum_{m=1}^{M}\theta_m z_{im}=\sum_{m=1}^{M}\theta_m\sum_{j=1}^{p}\phi_{mj}x_{ij}=\sum_{j=1}^{p}\sum_{m=1}^{M}\theta_m\phi_{mj}x_{ij}=\sum_{j=1}^{p}\beta_jx_{ij}, $$
where,
$$\beta_j=\sum_{m=1}^{M}\theta_m\phi_{mj}.$$
It helps me to think of this as a linear combination of linear combinations. It is important to note that this is still a linear model, and can be thought of as a special case of the original linear regression model. The only difference is that the coefficients are restrained to the form $\theta_m\phi_{mj}$.
With these restrained coefficients, it is possible to achieve a model with less bias and less variance than the traditional OLS model. It's a win-win!

#### Principal Components Regression
1. Compute the principal component (the normalized linear combination of the variables with the largest variance)
2. Compute the second principal component (which has the largest variance subject to being uncorrelated to the first)
3. And so on.

Hence, with many uncorrelated variables, we replace them with a small set of principal components that capture their joint variation.
We identify these principle components by looking solely at our feature matrix $X$, disregarding the target vector $y$. For this reason, these principle components (linear combinations) are identified in an *unsupervised* way. Inherently, PCR assumes that the variables with the largest variance are strong predictors of the target variable. This is often true, but not always the case.

In [103]:
#PCR to predict salary
pca = PCA()
X_train_reduced = pca.fit_transform(scaler.fit_transform(X_train))
regr = skl_lm.LinearRegression()        # We will use OLS to fit our M-dimensional data, derived from PCA

# 10-fold CV, with shuffle
kf_10 = KFold(n_splits=10, shuffle=False, random_state=1)
mse = []

for i in np.arange(1, 20):
    score = -1*cross_val_score(regr, X_train.values[:,:i], y_train, cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)
    
    
mse_per_component = pd.Series(np.array(mse).flatten(), index = np.arange(1,20))
print(mse_per_component)

1     187722.760108
2     190895.470470
3     187952.620187
4     191288.431157
5     191106.901317
6     174100.187213
7     172307.316013
8     162046.832194
9     160028.835931
10    173438.191930
11    168420.285339
12    169267.755551
13    168482.151098
14    168621.951795
15    169485.283815
16    162748.143066
17    166152.508018
18    168582.561441
19    169695.115601
dtype: float64


In [97]:
np.amin(mse_per_component)
# This lets us know that the model with 9 principal components has the smallest training error

160028.83593088179

##### Transform test data with PCA loadings and fit regression on 6 principal components

In [112]:
# optimal_pca = PCA(n_components=9)
X_test_reduced = pca.transform(scaler.fit_transform(X_test))[:, :7]

# train OLS model on PCA-reduced training data
pca_regr = skl_lm.LinearRegression()
pca_regr.fit(X_train_reduced[:,:7], y_train.values.ravel())

# Make predictions with our model and calculate MSE
y_pred = pca_regr.predict(X_test_reduced)
mean_squared_error(y_test.values.ravel(), y_pred)

96320.020782503241

#### Partial Least Squares
Unlike PCR, PLS identifies its linear combinations is a *supervised* way, making use of the response $y$ in order to identify new features that not only approximate the old features well, but also are related to the response.
In practice, PLS generally does not have much of an advantage over PCR

In [115]:
pls = PLSRegression()
pls.fit(scaler.fit_transform(X_train), y_train)

mean_squared_error(y_test, pls.predict(scaler.fit_transform(X_test)))

102234.27995999219